In [1]:

from datasets import load_from_disk, load_dataset
import numpy as np

/home/moonlab/decision_transformer/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_from_disk('/home/moonlab/decision_transformer/decision_tr/data/')

In [7]:
len(dataset['train']['returns_to_go'])

50

In [18]:
dataset['train'][0]['returns_to_go']

[[0.4875820577144623],
 [1.240638017654419],
 [2.4186158180236816],
 [3.100947856903076],
 [3.662254810333252],
 [4.363141059875488],
 [5.060859680175781],
 [6.156332015991211],
 [8.050374031066895],
 [10.362813949584961],
 [13.130672454833984],
 [15.875865936279297],
 [19.95923614501953],
 [24.876211166381836],
 [29.484722137451172],
 [33.26036071777344],
 [35.39275360107422],
 [38.16207504272461],
 [41.18669128417969],
 [43.085296630859375],
 [45.09266662597656],
 [47.86289596557617],
 [51.45220184326172],
 [54.88237380981445],
 [59.341888427734375],
 [65.59598541259766],
 [69.6114273071289],
 [73.71035766601562],
 [76.7156982421875],
 [80.77030181884766],
 [84.00093841552734],
 [86.666748046875],
 [89.04666137695312],
 [92.27933502197266],
 [96.20137786865234],
 [101.64672088623047],
 [105.72649383544922],
 [108.56980895996094],
 [111.78706359863281],
 [114.98662567138672],
 [117.65963745117188],
 [120.08843231201172],
 [121.5818862915039],
 [122.35489654541016],
 [123.3480377197265

In [4]:
ft = dataset['train']

In [11]:
len(ft['states'][0][0])

6

In [9]:
import numpy as np

states = []
for traj in ft['states']:
    states.append(traj)

# Concatenate all state arrays into a single array
states_concatenated = np.concatenate(states, axis=0)
print("States Concatenated Shape:", states_concatenated.shape)

# Calculate the mean and standard deviation
state_mean = np.mean(states_concatenated, axis=0)
state_std = np.std(states_concatenated, axis=0) + 1e-6  # Adding a small value to avoid division by zero

print("State Mean:", state_mean)
print("State Standard Deviation:", state_std)


States Concatenated Shape: (5000000, 6)
State Mean: [14.4524924 14.460679  14.4493306 14.5316842 14.4476096 14.4861036]
State Standard Deviation: [8.48190325 8.46530618 8.47361668 8.42939852 8.49589279 8.45907573]


In [7]:
feature = dataset['train'][0]

In [10]:
len(feature['states'])

1000

In [58]:
feature['timesteps'][1: 1+10]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [42]:
len(dataset['train'][0]['rewards'])

1000

In [18]:
len(dataset['train']['states'])

5000

In [12]:
dataset_1 = load_dataset("edbeeching/decision_transformer_gym_replay", "halfcheetah-expert-v2")

In [13]:
dataset_1

DatasetDict({
    train: Dataset({
        features: ['observations', 'actions', 'rewards', 'dones'],
        num_rows: 1000
    })
})

1000

In [19]:
states = []
traj_lens = []
for obs in dataset['train']["states"]:
    states.extend(obs)
    traj_lens.append(len(obs))
len(traj_lens)

5000

In [16]:
len(dataset_1['train'][0]['observations'])

1000

In [40]:
batch_indices = np.random.choice(a=np.arange(100), size=32, replace=False)
batch_indices

array([63, 54,  1, 47, 65, 23, 18,  9, 95,  7, 13, 92, 32, 64, 96,  5, 15,
       12, 58, 61,  2, 55, 78, 79, 71, 28, 44, 20, 99, 38, 35, 76])

In [8]:
from decision_tr.src.make_data.data_collator import DataCollate

In [25]:
data = DataCollate(batch_size=8, max_len=10, max_episode_len=10, num_traj=50, device='cuda')

In [26]:
d = data.make_batch()

From data collator:  state:  torch.Size([8, 10, 6]) action:  torch.Size([8, 10, 6]) torch.Size([8, 10, 1]) returns:  torch.Size([8, 10, 1]) torch.Size([8, 10]) torch.Size([16, 10])


In [27]:
d['returns_to_go'].shape

torch.Size([8, 10, 1])

In [22]:
returns = d['returns_to_go']
returns = returns.squeeze(-1)
print(returns.shape)

torch.Size([5, 10])


In [23]:
import torch.nn as nn

In [24]:
embed_returns = nn.Linear(10, 128).to('cuda')
returns_embed = embed_returns(returns).to('cuda')
print(returns_embed.shape) 

torch.Size([5, 128])


In [3]:
from datasets import load_from_disk

dataset = load_from_disk('/home/moonlab/decision_transformer/data')

In [4]:
import torch
model = torch.load('/home/moonlab/decision_transformer/decision_tr/saved_models/trained_model.pt')

/tmp/ipykernel_818574/3598151461.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('/home/moonlab/decision_transformer/decision_tr/saved_models/trained_

odict_items